# 概要
モーニングスターというサイトから全銘柄のコードと名称を取得するコード

In [160]:
# -*- coding: utf-8 -*-
import requests
from datetime import datetime, timedelta
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
import pickle
import warnings
warnings.filterwarnings('ignore')

In [161]:
def table_page(arg):
    page, flg = arg
    url = 'http://www.morningstar.co.jp/StockInfo/sec/list?page={}'.format(page)
    response = requests.get(url)
    soup = BeautifulSoup(response.content,"lxml")
    td = soup.body.find_all('td', class_ = 'tac')
    if len(td) != 0:
        code = [td[3*i].text for i in range(len(td)//3)]
        name = [td[3*i + 1].text for i in range(len(td)//3)]
        market = [td[3*i + 2].text for i in range(len(td)//3)]

        df = pd.DataFrame(columns = ['code','name','market'])
        df['code'] = code
        df['name'] = name
        df['market'] = market
    else:
        df = None
        flg = False
    return df, flg

## 並列処理のコード

In [162]:
#並列処理で取得
page_num = 82
args = [[i, True] for i in range(page_num)]
with ProcessPoolExecutor(max_workers = 8) as p:
    ret = p.map(table_page,args)
dfs = [r[0] for r in ret]

In [163]:
df_code = pd.concat(dfs, axis = 0).reset_index(drop = True)

In [169]:
#kabukaフォルダに全銘柄の名称とコードをcsvで出力
df_code.set_index('code').to_csv('kabuka/all_code.csv')

In [168]:
df_code

,code,name,market
0,1301,極洋,東証１部
1,1305,ダイワＴＰＸ,東証２部
2,1306,ＴＰＸ連動,東証２部
3,1308,上場ＴＰＸ,東証２部
4,1309,上証５０連動,東証２部
5,1310,コア３０,東証２部
6,1311,コア３０連動,東証２部
7,1312,ＲＮ小型コア,東証２部
8,1313,ＫＯＤＥＸ,東証２部
9,1319,３００投信,東証２部


## 通常処理のコード

In [165]:
# dfs = []
# page = 0
# flg = True
# while flg:
#     if page%10 == 0:
#         print(str(page) +' page finished')
#     df, flg = table_page([page, flg])
#     dfs.append(df)
#     page +=1
# print('finished!')

## サンプルコード

In [41]:
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor
def sample(args):
    a, b = args
    x = a + b
    return x

sample_a = np.arange(100)
sample_b = np.random.randint(0,100,100)
args = [[sample_a[i],sample_b[i]] for i in range(len(sample_a))]

with ProcessPoolExecutor(max_workers = 8) as p:
    ret = p.map(sample,args)